In [ ]:
import ssl
import sys
sys.path.append('./mcunet')
from mcunet.model_zoo import net_id_list, build_model, download_tflite
import os
from PIL import Image
import torch


# quant kd_mcunet
model_path = './imagenet_kd_proxy_224.pth'
model, image_size, description = build_model(net_id="proxyless-w0.3", pretrained=False)  # you can replace net_id with any other option from net_id_list
model.classifier = torch.nn.Identity()
model.classifier = torch.nn.Sequential(torch.nn.Linear(384,384))
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.classifier = torch.nn.Identity()
print(net_id_list)
print(model)

In [ ]:
import torch
from PIL import Image
from torchvision import transforms

img = Image.open('./PlantCLEF_Subset/train/ash/222064.jpg')

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img = preprocess(img)
img = img.unsqueeze(0) 

model.eval() 
output = model(img)
print(output)

In [ ]:
dummy_input = torch.randn(1,3,224,224)
input_names = ['input']
outout_names = ['output']

torch.onnx.export(
    model,
    dummy_input,
    './kd_proxy_224.onnx',
    do_constant_folding=False, # whether to execute constant folding for optimization
    input_names=input_names,
    output_names=outout_names,
    opset_version=11,
    export_params=True,
    verbose=False
)

In [ ]:
# onnx output
import onnxruntime
import numpy as np

img_np = img.numpy()

sess = onnxruntime.InferenceSession("./kd_proxy_224.onnx")

input_name = sess.get_inputs()[0].name
input_shape = sess.get_inputs()[0].shape


if len(input_shape) == 4: 
    img_np = img_np.reshape(input_shape)

output = sess.run(None, {input_name: img_np})

print(output)

In [ ]:
! onnx2tf -i ./kd_proxy_224.onnx -o ./quant_model -oiqt

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
import numpy as np
import tensorflow as tf
img = Image.open('./PlantCLEF_Subset/train/ash/222064.jpg')

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img = preprocess(img)
img = img.unsqueeze(0) 

img_np = img.numpy()

# for TensorFlow Lite，change (N, C, H, W) to (N, H, W, C)
img_np = np.transpose(img_np, (0, 2, 3, 1))

interpreter = tf.lite.Interpreter(model_path="./quant_model/kd_proxy_224_quant.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
img_np = img_np.astype(input_details[0]['dtype']).reshape(input_shape)

interpreter.set_tensor(input_details[0]['index'], img_np)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])

print(output_data)
